In [181]:
!ls TextSummary/

2170_001.txt              2172_001.txt              2175_001.txt
2170_001.txt_corecent.txt 2173_001.txt              2176_001.txt
2171_001.txt              2174_001.txt              2177_001.txt


In [184]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
 

 
 
# getting text document from file
fname="TextSummary/2173_001.txt"
with open(fname, 'r') as myfile:
      text=myfile.read()
     
     

  
print ('Summary:')
print (summarize(text, ratio=0.02))
 
print ('\nKeywords:')
print (keywords(text, ratio=0.01))
 


Summary:
Question 6 [last update 16 November 2016]: An AIF is marketed in a host Member State by 
Question 1 [last update 16 December 2016]: When a non-EU AIFM reports information to 
Answer 1: When a non-EU AIFM reports information to the national competent authorities of 
Question 3 [last update 25 March 2014]: Which period should AIFMs use when reporting 
Answer 8: AIFMs should aggregate the market value of all securities traded and report the 
Question  11  [last  update  25  March  2014]:  How  should  AIFMs  report  the  information  on 
Question  11  [last  update  25  March  2014]:  How  should  AIFMs  report  the  information  on 
Question 13 [last update 25 March 2014]: Should AIFMs report the information in English or 
Question  20 [last  update  27  June  2014]:  According  to  Article  24(2)  of the  AIFMD,  AIFMs 
reporting template for AIF-specific information) or value of turnover in each asset class over 
Question 25 [last update 21 July 2014]: AIFMs have to report val

In [174]:
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

# getting text document from file
fname="TextSummary/2170_001.txt"
with open(fname, 'r') as myfile:
      text=myfile.read()
     
sentences = nltk.sent_tokenize(text)
sentences = [sent.lower() for sent in sentences]
print(len(sentences))
tokenizer = RegexpTokenizer(r'\w+')
sentences = [tokenizer.tokenize(sent) for sent in sentences]
print(len(sentences))
sentences = [" ".join(sent) for sent in sentences if len(sent)>20]
print(len(sentences))

# text = ''.join(sent for sent in sentences)

# fname="TextSummary/2170_001.txt"
# with open(fname, 'r') as myfile:
#       text=myfile.read()
     

# file = open(f'{fname}_corecent.txt',"w")
# file.write(str(sentences))
# file.close()
#Copy and paste...will work.
#     tokens = tokenizer.tokenize(sentence)
#     filtered_words = [w for w in tokens if not w in stopwords.words('english')]
#     return " ".join(filtered_words)

print ('Summary:')
print (summarize(text, ratio=0.02))
 
# print ('\nKeywords:')
# print (keywords(text, ratio=0.01))

224
224
181
Summary:
Delayed inside information  Article 17(4) of MAR   
Delayed disclosure of inside  Article 17(5) of MAR   
Delayed disclosure of inside  Article 17(5) of MAR   
Delayed disclosure of inside  Article 17(5) of MAR   
Article  8(1)  of  MAR  states  that  “the  use  of  inside  information  by  cancelling  or 
Where the issuer is also a credit or financial institution, Article 17(5) of MAR allows 
disclosure  of  inside  information  under  Article  17(4)  of  MAR  or  notifying  to  the 
inside  information  in  accordance  with  Article  17(4)  of  MAR  and,  due  to 
According  to  Article  17(1)  of  MAR,  an  issuer  has  to  inform  the  public  as  soon  as 
Where an issuer has delayed the disclosure of inside information according to Article 
Where the issuer has delayed the disclosure of inside information in accordance with 
Delayed disclosure of inside information under Article 17(5) of MAR: assessment of the 
inside  information  under  Article  17(5)  of  

In [122]:
#f = open("test1.txt", "w")
with open('test1.txt', 'w') as f:
    for i in sentences:
        print(i, file=f) 
#f.close()  

In [127]:
fname="Test1.txt"
with open(fname, 'r') as myfile:
      text=myfile.read()
        
type(text)
print ('Summary:')
print (summarize(sentences, ratio=0.02))

Summary:


TypeError: expected string or bytes-like object

In [171]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from string import punctuation
from heapq import nlargest

class FrequencySummarizer:
  def __init__(self, min_cut=0.2, max_cut=0.8):
    """
     Initilize the text summarizer.
     Words that have a frequency term lower than min_cut 
     or higer than max_cut will be ignored.
    """
    self._min_cut = min_cut
    self._max_cut = max_cut 
    self._stopwords = set(stopwords.words('english') + list(punctuation))

  def _compute_frequencies(self, word_sent):
    """ 
      Compute the frequency of each of word.
      Input: 
       word_sent, a list of sentences already tokenized.
      Output: 
       freq, a dictionary where freq[w] is the frequency of w.
    """
    freq = defaultdict(int)
    for s in word_sent:
      for word in s:
        if word not in self._stopwords:
          freq[word] += 1
    # frequencies normalization and fitering
    m = float(max(freq.values()))
    #for w in freq.keys():
    for w in list(freq):
      freq[w] = freq[w]/m
      if freq[w] >= self._max_cut or freq[w] <= self._min_cut:
        del freq[w]
    return freq

  def summarize(self, text, n):
    """
      Return a list of n sentences 
      which represent the summary of text.
    """
    sents = sent_tokenize(text)
    assert n <= len(sents)
    word_sent = [word_tokenize(s.lower()) for s in sents]
    self._freq = self._compute_frequencies(word_sent)
    ranking = defaultdict(int)
    for i,sent in enumerate(word_sent):
      for w in sent:
        if w in self._freq:
          ranking[i] += self._freq[w]
    sents_idx = self._rank(ranking, n)    
    return [sents[j] for j in sents_idx]

  def _rank(self, ranking, n):
    """ return the first n sentences with highest ranking """
    return nlargest(n, ranking, key=ranking.get)


In [190]:
fs = FrequencySummarizer()
for s in fs.summarize(text, 2):
   print(s)
# fs = FrequencySummarizer()
# fs(sentences)


In  line  with  the  approach  followed  under  the  AIFMD  Remuneration 
Guidelines9, AIFMs can ensure compliance in one of the following two ways: 
i) 
ii) 
where  the  delegate  is  subject  to  regulatory  requirements  on  remuneration 
disclosure for its staff to whom portfolio management or risk management activities 
have been delegated that are equally as effective as those under Article 22(2)(e) of 
the AIFMD, the AIFM should use the information disclosed by the delegate for the 
purposes of fulfilling its obligations under Article 22(e) of the AIFMD and Article 107 
of the AIFMD Level 2 Regulation; or  
in other cases, appropriate contractual arrangements should be put in place with 
the delegate allowing the AIFM to receive (and disclose in the annual report for the 
relevant  AIF(s)  that  it  manages)  at  least  information  on  the  total  amount  of 
remuneration for the financial year, split into fixed and variable remuneration, paid 
by the AIF and/or the AIFM to the

In [145]:
# Simple Summarizer
# Copyright (C) 2010-2012 Tristan Havelick
# Author: Tristan Havelick <tristan@havelick.com>
# URL: <https://github.com/thavelick/summarize/>
# For license information, see LICENSE.TXT

"""
A summarizer based on the algorithm found in Classifier4J by Nick Lothan.
In order to summarize a document this algorithm first determines the
frequencies of the words in the document.  It then splits the document
into a series of sentences.  Then it creates a summary by including the
first sentence that includes each of the most frequent words.  Finally
summary's sentences are reordered to reflect that of those in the original
document.
"""

##//////////////////////////////////////////////////////
##  Simple Summarizer
##//////////////////////////////////////////////////////

from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk.data

class SimpleSummarizer:

	def reorder_sentences( self, output_sentences, input ):
		output_sentences.sort( lambda s1, s2:
			input.find(s1) - input.find(s2) )
		return output_sentences
	
	def get_summarized(self, input, num_sentences ):
		# TODO: allow the caller to specify the tokenizer they want
		# TODO: allow the user to specify the sentence tokenizer they want
		
		tokenizer = RegexpTokenizer('\w+')
		
		# get the frequency of each word in the input
		base_words = [word.lower()
			for word in tokenizer.tokenize(input)]
		words = [word for word in base_words if word not in stopwords.words()]
		word_frequencies = FreqDist(words)
		
		# now create a set of the most frequent words
		most_frequent_words = [pair[0] for pair in
			word_frequencies.items()[:100]]
		
		# break the input up into sentences.  working_sentences is used
		# for the analysis, but actual_sentences is used in the results
		# so capitalization will be correct.
		
		sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
		actual_sentences = sent_detector.tokenize(input)
		working_sentences = [sentence.lower()
			for sentence in actual_sentences]

		# iterate over the most frequent words, and add the first sentence
		# that inclues each word to the result.
		output_sentences = []

		for word in most_frequent_words:
			for i in range(0, len(working_sentences)):
				if (word in working_sentences[i]
				  and actual_sentences[i] not in output_sentences):
					output_sentences.append(actual_sentences[i])
					break
				if len(output_sentences) >= num_sentences: break
			if len(output_sentences) >= num_sentences: break
			
		# sort the output sentences back to their original order
		return self.reorder_sentences(output_sentences, input)
	
	def summarize(self, input, num_sentences):
		return " ".join(self.get_summarized(input, num_sentences))

In [186]:
summerizer = SimpleSummarizer()
for sentence in summarizer(parser.document, 1):
        print(sentence)

print(parser.document)

(systems, templates and records) templates to be used by disclosing market participants  and the format of the records in accordance with Regulation (EU) No 596/2014 of the European Parliament and of the Council (Text with EEA relevance)8 RTS on accepted market practices Commission  Delegated  Regulation  (EU)  2016/908  of  26 February 2016 supplementing Regulation (EU) No 596/2014 of the European Parliament and of the Council laying down regulatory technical standards on the criteria, the procedure and  the  requirements  for  establishing  an  accepted  market practice and the requirements for maintaining it, terminating it  or  modifying  the  conditions  for  its  acceptance  (Text  with EEA relevance) 9 RTS on the prevention, detection and reporting of abusive practices or suspicious orders or transactions Commission  Delegated  Regulation  (EU)  2016/957  of  9 March 2016 supplementing Regulation (EU) No 596/2014 of the  European  Parliament  and  of  the  Council  with  regard 

In [86]:
import sumy

In [188]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
import sumy
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers import text_rank
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer   #found this is the best as 
# it is picking from beginning also while other skip
 
 
LANGUAGE = "english"
SENTENCES_COUNT = 10
stemmer = Stemmer(LANGUAGE)
 
if __name__ == "__main__":
    
#     url="https://en.wikipedia.org/wiki/Deep_learning"
   
#     parser = HtmlParser.from_url(url, Tokenizer(LANGUAGE))
    # or for plain text files
    parser = PlaintextParser.from_file("TextSummary/2170_001.txt", Tokenizer(LANGUAGE))
    
    print ("--Lexrank--")    
    summarizer = ()
    summarizer = LexRankSummarizer(Stemmer(LANGUAGE))
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
        
    print ("--sumbasic--")    
    summarizer = SumBasicSummarizer(stemmer)
    summarizer = SumBasicSummarizer(Stemmer(LANGUAGE))
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
        
    print ("--LsaSummarizer--")    
    summarizer = LsaSummarizer()
    summarizer = LsaSummarizer(Stemmer(LANGUAGE))
    summarizer.stop_words = get_stop_words(LANGUAGE)
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
         
    print ("--LuhnSummarizer--")     
    summarizer = LuhnSummarizer() 
    summarizer = LsaSummarizer(Stemmer(LANGUAGE))
    summarizer.stop_words = ("I", "am", "the", "you", "are", "me", "is", "than", "that", "this",)
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
         
    print ("--EdmundsonSummarizer--")     
    summarizer = EdmundsonSummarizer() 
    words = ("mar", "investment", "financial" )
    summarizer.bonus_words = words
     
    words = ("another", "and", "some", "next",)
    summarizer.stigma_words = words
    
     
    words = ("another", "and", "some", "next",)
    summarizer.null_words = words
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)   

words = [word for word in tokens if word.isalpha()]

--Lexrank--
1 OJ L 331, 15.12.2010, p. 84 2 OJ L 173, 12.6.2014, p. 1–61 3 OJ L 332, 18.12.2015, p. 126–132 3 Delegated regulation on an exemption for certain third countries public bodies and central banks, the indicators of market manipulation, the disclosure thresholds, the competent authority for notifications of delays, the permission for trading during closed periods and types of notifiable managers' transactions RTS on financial instrument reference data under Article 4 of
According  to  Article  7  of  MAR,  inside information is such information that is:
According  to  Article  17(1)  of  MAR,  an  issuer  has  to  inform  the  public  as  soon  as possible of inside information that directly concerns that issuer.
Where an issuer has delayed the disclosure of inside information according to Article 17(4) of MAR, immediately after the information is disclosed to the public the issuer needs  to  inform  the  competent  authority  that  disclosure  of  inside  information  was de

NameError: name 'tokens' is not defined

In [94]:
len(get_stop_words(LANGUAGE))

580

In [143]:
parser.document

<DOM with 288 paragraphs>